In [1]:
from langgraph.graph import StateGraph, START, END
#import google chat model
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluation: str

In [5]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']
    prompt = f'Generate a detailed outline for a blog on the - {title}'
    outline = model.invoke(prompt).content
    state['outline'] = outline

    return state


In [6]:
def create_blog(state: BlogState) -> BlogState:
    
    title = state['title']
    outline = state['outline']
    prompt = f'Write a detailed blog the title - {title} using the following outline \n {outline}'
    content = model.invoke(prompt).content
    state['content'] = content
    
    return state


In [7]:
def evaluate_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    content = state['content']
    prompt = (
        "You are a concise blog reviewer. Rate the blog content 1-10 based on how well it "
        "follows the outline while staying relevant to the title. Provide a one-line justification.\n"
        f"Title: {title}\n"
        f"Outline: {outline}\n"
        f"Blog: {content}"
    )
    state['evaluation'] = model.invoke(prompt).content
    return state


In [8]:
graph = StateGraph(BlogState)

# nodes 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# Edges

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [9]:
intial_state = {'title': 'Rise of AI in Pakistan'}
final_state = workflow.invoke(intial_state)
print(final_state)

{'title': 'Rise of AI in Pakistan', 'outline': 'Okay, this is a fascinating and timely topic! Here\'s a detailed outline for a blog post on the "Rise of AI in Pakistan," designed to be comprehensive, engaging, and informative.\n\n---\n\n## Blog Title Options:\n\n*   The AI Awakening: Charting the Rise of Artificial Intelligence in Pakistan\n*   From Nascent to Noteworthy: Pakistan\'s Emergence in the Global AI Landscape\n*   Pakistan\'s AI Frontier: Opportunities, Challenges, and the Road Ahead\n*   The Digital Leap: How AI is Reshaping Pakistan\'s Future\n\n---\n\n## Target Audience:\n\n*   Tech enthusiasts, entrepreneurs, investors, policymakers, students, and anyone interested in Pakistan\'s technological advancement.\n\n## Tone:\n\n*   Informative, optimistic but realistic, analytical, and forward-looking.\n\n---\n\n## Detailed Blog Outline:\n\n### I. Introduction: Setting the Stage (Approx. 150-200 words)\n\n*   **A. The Global AI Revolution:**\n    *   Brief overview of AI\'s tra

In [11]:
print(final_state['outline'])

Okay, this is a fascinating and timely topic! Here's a detailed outline for a blog post on the "Rise of AI in Pakistan," designed to be comprehensive, engaging, and informative.

---

## Blog Title Options:

*   The AI Awakening: Charting the Rise of Artificial Intelligence in Pakistan
*   From Nascent to Noteworthy: Pakistan's Emergence in the Global AI Landscape
*   Pakistan's AI Frontier: Opportunities, Challenges, and the Road Ahead
*   The Digital Leap: How AI is Reshaping Pakistan's Future

---

## Target Audience:

*   Tech enthusiasts, entrepreneurs, investors, policymakers, students, and anyone interested in Pakistan's technological advancement.

## Tone:

*   Informative, optimistic but realistic, analytical, and forward-looking.

---

## Detailed Blog Outline:

### I. Introduction: Setting the Stage (Approx. 150-200 words)

*   **A. The Global AI Revolution:**
    *   Brief overview of AI's transformative impact worldwide (industry, daily life, economic growth).
    *   Ment

In [12]:
print(final_state['content'])

## The Digital Leap: How AI is Reshaping Pakistan's Future

---

### I. Introduction: Setting the Stage

The world stands at the precipice of an unprecedented technological transformation, largely driven by Artificial Intelligence (AI). From revolutionizing industries and streamlining daily tasks to accelerating economic growth, AI has become a global force. Major players like the U.S., China, and Europe are investing heavily, recognizing AI's potential to redefine societies. Amidst this global AI revolution, an unexpected yet rapidly emerging player is making its mark: Pakistan.

With a youthful demographic, a burgeoning digital literacy rate, and an increasingly connected population, Pakistan presents a fertile ground for the adoption and development of AI. This unique blend of human potential and technological readiness positions the nation at a pivotal moment. This blog post argues that Pakistan is experiencing a significant, albeit nascent, rise in AI adoption and development, dri

In [13]:
print(final_state['evaluation'])

**Rating:** 10/10

**Justification:** The blog meticulously follows the comprehensive outline, addressing every section and sub-point with relevant content and a consistent, appropriate tone.
